In [ ]:
import laspy
import numpy as np
import pandas as pd
import os

In [ ]:


file_path = r"C:\Users\eashenhurst\Desktop\local scans\test_csv_core\hyperspectral lights off\Scan Slot 1 No Hysperctral Lights_luminance.csv"
df = pd.read_csv(file_path, header=None)  

z = df.to_numpy().flatten().tolist()

x_axis_size = 0.25

y_axis_size = 0.225
x_values = np.arange(0, 10000) * x_axis_size
y_values = np.arange(-1599.5, 1600, 1) * y_axis_size

x = np.repeat(x_values, 3200) 
y = np.tile(y_values, 10000) 

folder = os.path.dirname(file_path)

file_path = os.path.join(folder, "intensity_cloud.bpc")

xyz = np.column_stack((x, y, z)).astype(np.float32)

xyz.tofile(file_path)

print(f"BPC file written: {file_path}")

In [ ]:


file_path = os.path.join(folder, "intensity_cloud.las")

header = laspy.LasHeader(point_format=1, version="1.2") 
las = laspy.LasData(header)

las.x = np.array(x)
las.y = np.array(y)
las.z = np.array(z)

las.write(file_path)